In [ ]:


## Code for running Wizardry-esque early dungeon crawl games
## by Nicholas Smith
## Copyright (c) 2025

In [ ]:
## import statements go here
import nbformat
import importlib
import import_ipynb
import io
import os
import sys
import pygame

In [ ]:


## Global constants

WALL    = 0b0
ROOM    = 0b1
VISITED = 0b10
PLAYER  = 0b100
ENEMY   = 0b1000
ROOMKEY = 0b10000

## Global variables

mapWidth = 0
mapHeight = 0
currentmap = None                                   # Current level map
savefile = None                                     # Current loaded save file
playerfacing = 0                                    # Direction player is facing (1 for North, 2 for East, 3 for South, 4 for West)
enemies = []                                        # List of all enemy Things
items = []                                          # List of all item Things
VALID_FAMILIES = ['player','enemy','item']          # list of all Thing families in game
VALID_TYPES = ['player','enemy','item']             # List of all Thing types
screen = None                                       # pygame display surface
pointsone = "TEST"                                  # Points denoting far corners of player's space on the map
pointstwo = None                                    # Points denoting far corners of space one in front of player on the map
pointsthree = None                                  # Points denoting far corners of space two in front of player on the map
pointsfour = None                                   # Points denoting far corners of space three in front of player on the map
screenWidth = 0                                     # Width of pygame window in pixels
screenHeight = 0                                    # Height of pygame window in pixels

In [ ]:


## Main launch method

def main():
    global currentmap
    global playerfacing

    # Initialize the pygame window
    initialize()
    
    # Load up the first map
    currentmap = loadMap("MovementTestLevel.lvlmap")

    # Ensure player is pointing North
    playerfacing = 1

    # Draw a first-person view of the initial map state
    drawView(currentmap)

    # Begin the event-capture game loop
    gameLoop()
    # print("All good")
    return

In [ ]:


## Load current map into memory from disk
## Returns a Map object

def loadMap(mapfile):

    global mapWidth
    global mapHeight

    mapstream = open(mapfile, "r", encoding="utf-8")
    try:
        # Ensure the filetype is correct
        teststring = mapstream.readline()
        if teststring != "Wizdrive map\n":
            raise ValueError("Loaded file was not a valid map.")

        # Split map into a two-dimensional array
        map = mapstream.read()
        map = map.split("\n")
        map[len(map)-1].strip()
        for l, line in enumerate(map):
            map[l] = line.split(",")
    
    # Set mapHeight and mapWidth based on this new array
        mapHeight = len(map)
        mapWidth = len(map[0])
    # Sanity check; all rows must be the length of mapWidth and there must be mapHeight rows
        for l in range(mapHeight):
            for c in range(mapWidth):
                testvar = map[l][c]
                map[l][c] = int(map[l][c])
    except Exception:
        sys.exit(1)
    return map

In [ ]:


## Main game loop

def gameLoop():
    while True:
        # Poll continuously for keyboard input
        # When one is found, parse it and perform the action
        # for gameEvent in pygame.event.get():
        for gameEvent in pygame.event.get():
            if gameEvent.type == pygame.KEYDOWN:
                print("Key pressed")
                keyboardHandler(gameEvent.key)
            
    # Enemies take turns
    # Repeat
    return

In [ ]:


##################################### USER CONTROL CODE #######################################



In [ ]:
## Handle keyboard input

def keyboardHandler(pressed):
    print("keyboardHandler works")
    match pressed:
        case pygame.K_UP:
            print("K_UP pressed")
        case _:
            print("Not K_UP")
    return
keyboardHandler("no")

In [ ]:


###################################### GRAPHICS CODE #######################################



In [ ]:

## Initialize pygame and set graphics constants based on size of window

def initialize():
    # Method uses all global graphics variables
    global screen
    global pointsone
    global pointstwo
    global pointsthree
    global pointsfour
    global screenWidth
    global screenHeight

    # initialize Pygame
    pygame.init()
    (screenWidth, screenHeight) = (800, 800)
    pointsone = ((screenWidth*.20, screenHeight*.20), (screenWidth*.80, screenHeight*.20), (screenWidth*.2, screenHeight*.8),(screenWidth*.8, screenHeight*.8))
    pointstwo = ((screenWidth*.4, screenHeight*.40), (screenWidth*.60, screenHeight*.40), (screenWidth*.4, screenHeight*.6),(screenWidth*.6, screenHeight*.6))
    pointsthree = ((screenWidth*.45, screenHeight*.45), (screenWidth*.55, screenHeight*.45), (screenWidth*.45, screenHeight*.55),(screenWidth*.55, screenHeight*.55))
    pointsfour = ((screenWidth*.455, screenHeight*.455), (screenWidth*.525, screenHeight*.455), (screenWidth*.455, screenHeight*.525),(screenWidth*.525, screenHeight*.525))

    screen = pygame.display.set_mode(size=(screenWidth, screenHeight))
    screen.fill("purple")
    pygame.display.flip()

In [ ]:

## Draw cell for currently occupied space

def __drawZero():
    pygame.draw.line(screen, "black", (0, 0), pointsone[0])
    pygame.draw.line(screen, "black", (0, screenHeight), pointsone[2])
    pygame.draw.line(screen, "black", (screenWidth, 0), pointsone[1])
    pygame.draw.line(screen, "black", (screenWidth, screenHeight), pointsone[3])
    pygame.draw.line(screen, "black", pointsone[0], pointsone[1])
    pygame.draw.line(screen, "black", pointsone[0], pointsone[2])
    pygame.draw.line(screen, "black", pointsone[1], pointsone[3])
    pygame.draw.line(screen, "black", pointsone[2], pointsone[3])
    return

## Draw cell to left of player
def __drawZeroLeft():
    pygame.draw.line(screen, "black", pointsone[0], (0, pointsone[0][1]))
    pygame.draw.line(screen, "black", pointsone[2], (0, pointsone[2][1]))
    return

## Draw cell to right of player
def __drawZeroRight():
    pygame.draw.line(screen, "black", pointsone[1], (screenWidth, pointsone[1][1]))
    pygame.draw.line(screen, "black", pointsone[3], (screenWidth, pointsone[3][1]))
    return

## Draw cell one space ahead of player
def __drawOne():
    pygame.draw.line(screen, "black", pointsone[0], pointstwo[0])
    pygame.draw.line(screen, "black", pointsone[2], pointstwo[2])
    pygame.draw.line(screen, "black", pointsone[1], pointstwo[1])
    pygame.draw.line(screen, "black", pointsone[3], pointstwo[3])
    pygame.draw.line(screen, "black", pointstwo[0], pointstwo[1])
    pygame.draw.line(screen, "black", pointstwo[0], pointstwo[2])
    pygame.draw.line(screen, "black", pointstwo[1], pointstwo[3])
    pygame.draw.line(screen, "black", pointstwo[2], pointstwo[3])
    return

## Draw cell one left
def __drawOneLeft():
    pygame.draw.line(screen, "black", pointstwo[0], (pointsone[0][0], pointstwo[0][1]))
    pygame.draw.line(screen, "black", pointstwo[2], (pointsone[2][0], pointstwo[2][1]))
    return

## Draw cell one right
def __drawOneRight():
    pygame.draw.line(screen, "black", pointstwo[1], (pointsone[1][0], pointstwo[1][1]))
    pygame.draw.line(screen, "black", pointstwo[3], (pointsone[3][0], pointstwo[3][1]))
    return

## Draw cell two spaces ahead of player
def __drawTwo():
    pygame.draw.line(screen, "black", pointstwo[0], pointsthree[0])
    pygame.draw.line(screen, "black", pointstwo[2], pointsthree[2])
    pygame.draw.line(screen, "black", pointstwo[1], pointsthree[1])
    pygame.draw.line(screen, "black", pointstwo[3], pointsthree[3])
    pygame.draw.line(screen, "black", pointsthree[0], pointsthree[1])
    pygame.draw.line(screen, "black", pointsthree[0], pointsthree[2])
    pygame.draw.line(screen, "black", pointsthree[1], pointsthree[3])
    pygame.draw.line(screen, "black", pointsthree[2], pointsthree[3])
    return

## Draw cell two left
def __drawTwoLeft():
    pygame.draw.line(screen, "black", pointsthree[0], (pointstwo[0][0], pointsthree[0][1]))
    pygame.draw.line(screen, "black", pointsthree[2], (pointstwo[2][0], pointsthree[2][1]))
    return

## Draw cell two right
def __drawTwoRight():
    pygame.draw.line(screen, "black", pointsthree[1], (pointstwo[1][0], pointsthree[1][1]))
    pygame.draw.line(screen, "black", pointsthree[3], (pointstwo[3][0], pointsthree[3][1]))
    return

In [ ]:


## Draw the first-person view of the current Player location and directional orientation
## based on a submitted local map

def drawView(localmap):
    __drawZero()
    if localmap[2][0]:
        __drawZeroLeft()
    if localmap[2][2]:
        __drawZeroRight()
    if localmap[1][1]:
        __drawOne()
        if localmap[1][0]:
            __drawOneLeft()
        if localmap[1][2]:
            __drawOneRight()
        if localmap[0][1]:
            __drawTwo()
            if localmap[0][0]:
                __drawTwoLeft()
            if localmap[0][2]:
                __drawTwoRight()
    pygame.display.flip()
    
    

In [ ]:


## Program launch code
if __name__ == '__main__':
    print("Pygame works")
    main()